In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0837,  0.3820,  0.0155,  0.4070, -0.2481, -0.1475, -0.4390,  0.5682,
         -0.2286,  0.0147,  0.2993,  0.0300,  0.1412,  0.1364, -0.0473,  0.3537,
          0.4868,  0.0619, -0.0240,  0.1940],
        [ 0.5084,  0.3326,  0.0572,  0.2951,  0.0581, -0.3091, -0.4482,  0.5843,
         -0.3048, -0.2782,  0.3386, -0.2071,  0.2470,  0.4147,  0.3438,  0.2189,
         -0.0726,  0.3182, -0.1637, -0.0494],
        [ 0.4212,  0.0033,  0.4566,  0.0283, -0.3090, -0.3909, -0.4641,  0.7562,
         -0.0375, -0.0290,  0.5361, -0.1383, -0.1478,  0.1085,  0.2607,  0.3462,
          0.3306, -0.1739, -0.2657,  0.3435]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0837, 0.3820, 0.0155, 0.4070, 0.0000, 0.0000, 0.0000, 0.5682, 0.0000,
         0.0147, 0.2993, 0.0300, 0.1412, 0.1364, 0.0000, 0.3537, 0.4868, 0.0619,
         0.0000, 0.1940],
        [0.5084, 0.3326, 0.0572, 0.2951, 0.0581, 0.0000, 0.0000, 0.5843, 0.0000,
         0.0000, 0.3386, 0.0000, 0.2470, 0.4147, 0.34

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)